<a href="https://colab.research.google.com/github/AlekseevaTatiana23/Machine-learning/blob/main/Attestation_HT11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 11. Качество модели и её улучшение
Тебе предстоит разработать модель машинного обучения для предсказания вероятности оттока клиентов в телекоммуникационной компании. У тебя есть набор данных, который содержит различные признаки о клиентах, такие как возраст, пол, тип подключения, длительность пользования услугами и т.д.

Твоя задача состоит в следующем:

1. Загрузить данные и провести предварительный анализ данных.

2. Подготовить данные для обучения модели. Это может включать удаление ненужных признаков, заполнение пропущенных значений и нормализацию данных.

3. Разделить данные на обучающую и тестовую выборки.

4. Выбрать модель машинного обучения, которую ты хочешь использовать для предсказания оттока клиентов. Например, можно использовать модель классификации, такую как логистическая регрессия или случайный лес.

5. Обучить модель на обучающей выборке и оценить ее производительность на тестовой выборке.

6. Провести анализ результатов и оценить важность различных признаков для предсказания оттока клиентов.

7. При необходимости, провести дополнительные эксперименты, чтобы улучшить производительность модели. Это может включать изменение параметров модели, добавление новых признаков или использование другой модели машинного обучения.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder





In [ ]:
# 1. Загрузка данных и предварительный анализ
data = pd.read_csv('telecom_churn.csv')
print(data.info())
print(data.describe())
print(data['Churn'].value_counts())

# Предположим, что признак 'Churn' — целевая переменная (1 — ушёл, 0 — остался)
# Посмотрим на пропуски
print(data.isnull().sum())






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   3333 non-null   object 
 1   Account length          3333 non-null   int64  
 2   Area code               3333 non-null   int64  
 3   International plan      3333 non-null   object 
 4   Voice mail plan         3333 non-null   object 
 5   Number vmail messages   3333 non-null   int64  
 6   Total day minutes       3333 non-null   float64
 7   Total day calls         3333 non-null   int64  
 8   Total day charge        3333 non-null   float64
 9   Total eve minutes       3333 non-null   float64
 10  Total eve calls         3333 non-null   int64  
 11  Total eve charge        3333 non-null   float64
 12  Total night minutes     3333 non-null   float64
 13  Total night calls       3333 non-null   int64  
 14  Total night charge      3333 non-null   

In [ ]:
# 2. Предварительная подготовка данных
# Удалим ненужные признаки (например, идентификаторы)
if 'customerID' in data.columns:
    data = data.drop('customerID', axis=1)

# Заполнение пропусков (если есть)
# Например, для числовых признаков
for col in data.select_dtypes(include=['float64', 'int64']).columns:
    data[col].fillna(data[col].median(), inplace=True)

# Для категориальных признаков
for col in data.select_dtypes(include=['object']).columns:
    data[col].fillna(data[col].mode()[0], inplace=True)

# Кодирование категориальных признаков
# Можно использовать LabelEncoder или OneHotEncoder
# Для простоты используем LabelEncoder
label_encoders = {}
for col in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Разделение признаков и целевой переменной
X = data.drop('Churn', axis=1)
y = data['Churn']




/tmp/ipython-input-37-278086553.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
/tmp/ipython-input-37-278086553.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [43]:
# 3. Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Масштабирование признаков (важно для некоторых моделей)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [42]:
# 4. Выбор модели — например, случайный лес
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 5. Обучение и оценка
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print("Classification report:\n", classification_report(y_test, y_pred))

Accuracy: 0.943
Classification report:
               precision    recall  f1-score   support

       False       0.95      0.99      0.97       570
        True       0.90      0.68      0.78        97

    accuracy                           0.94       667
   macro avg       0.93      0.83      0.87       667
weighted avg       0.94      0.94      0.94       667



Точность модели составляет примерно 94%, что говорит о хорошей предсказательной способности. В отчёте также видно, что модель хорошо распознает клиентов, которые уйдут (True), хотя есть небольшая разница в показателях recall и f1-score для этого класса.

In [ ]:
# 6. Анализ важности признаков
importances = model.feature_importances_
feature_names = X.columns if hasattr(X, 'columns') else [f'feature_{i}' for i in range(X.shape[1])]
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values(by='importance', ascending=False)
print(importance_df)



                   feature  importance
6        Total day minutes    0.150866
8         Total day charge    0.131086
18  Customer service calls    0.125301
3       International plan    0.072397
9        Total eve minutes    0.062197
11        Total eve charge    0.061623
16        Total intl calls    0.048309
15      Total intl minutes    0.042353
17       Total intl charge    0.037553
5    Number vmail messages    0.032786
7          Total day calls    0.032273
14      Total night charge    0.032154
12     Total night minutes    0.032087
1           Account length    0.030770
13       Total night calls    0.029843
10         Total eve calls    0.026097
0                    State    0.024460
4          Voice mail plan    0.020572
2                Area code    0.007272


Самыми важными являются:

Total day minutes (0.151)
Total day charge (0.131)
Customer service calls (0.125)
International plan (0.072)
Это говорит о том, что длительность дневных звонков, связанные с ними расходы, а также количество обращений в службу поддержки существенно влияют на вероятность ухода клиента.

In [ ]:
# 7. Улучшение модели — например, подбор гиперпараметров
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid.fit(X_train, y_train)

print(f"Лучшие параметры: {grid.best_params_}")
print(f"Лучшая точность по кросс-валидации: {grid.best_score_:.3f}")

# Обучение финальной модели с лучшими гиперпараметрами
best_model = grid.best_estimator_
best_model.fit(X_train, y_train)
final_pred = best_model.predict(X_test)
print(f"Точность финальной модели: {accuracy_score(y_test, final_pred):.3f}")
print("Финальный отчет:\n", classification_report(y_test, final_pred))



Лучшие параметры: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}
Лучшая точность по кросс-валидации: 0.957
Точность финальной модели: 0.940
Финальный отчет:
               precision    recall  f1-score   support

       False       0.95      0.99      0.97       570
        True       0.89      0.67      0.76        97

    accuracy                           0.94       667
   macro avg       0.92      0.83      0.87       667
weighted avg       0.94      0.94      0.94       667



Оптимальные параметры модели:

max_depth: без ограничения


min_samples_split: 5


n_estimators: 200


Это позволило достичь кросс-валидационной точности до 95.7%, что значительно превосходит исходную.

 Финальная модель показывает:

Общую точность около 94%


Высокий recall для класса False (99%), что важно для минимизации пропущенных случаев


Немного ниже recall для класса True (67%), что можно дополнительно улучшить, если нужно